# JRA Scraping Notebook

中央競馬（JRA）のデータをスクレイピングし、Google Drive上のデータセットに追加します。

In [1]:
# 1. Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

import os
import sys

# ★★★ 設定項目 ★★★
# scraperフォルダが存在するパス (Google Drive上のパス)
# 例: '/content/drive/MyDrive/dai-keiba'
PROJECT_PATH = '/content/drive/MyDrive/dai-keiba'

if not os.path.exists(PROJECT_PATH):
    print(f"Error: Path {PROJECT_PATH} does not exist. Please check your Drive structure.")
else:
    print(f"Project path found: {PROJECT_PATH}")
    os.chdir(PROJECT_PATH)
    sys.path.append(PROJECT_PATH)


Mounted at /content/drive
Project path found: /content/drive/MyDrive/dai-keiba


In [2]:
# 2. 必要なライブラリのインポート
try:
    import pandas as pd
    import requests
    import bs4
except ImportError:
    !pip install pandas requests beautifulsoup4
    import pandas as pd
    import requests
    import bs4

from datetime import datetime, date
from scraper.jra_scraper import scrape_jra_year, JRA_MONTH_PARAMS
import time

# ==========================================
# 🚨 2023年/2022年パラメータ不足の修正パッチ
# ==========================================
# パラメータマップを上書き更新
try:
    JRA_MONTH_PARAMS.update({
        "2023": { "01": "27", "02": "F5", "03": "C3", "04": "91", "05": "5F", "06": "2D", "07": "FB", "08": "C9", "09": "97", "10": "06", "11": "D4", "12": "A2" },
        "2022": { "01": "9B", "02": "69", "03": "37", "04": "05", "05": "D3", "06": "A1", "07": "6F", "08": "3D", "09": "0B", "10": "7A", "11": "48", "12": "16" },
        "2021": { "01": "0F", "02": "DD", "03": "AB", "04": "79", "05": "47", "06": "15", "07": "E3", "08": "B1", "09": "7F", "10": "EE", "11": "BC", "12": "8A" },
        "2020": { "01": "83", "02": "51", "03": "1F", "04": "ED", "05": "BB", "06": "89", "07": "57", "08": "25", "09": "F3", "10": "62", "11": "30", "12": "FE" }
    })
    print("✅ JRA 2023/2022 Parameters have been patched successfully.")
except NameError:
    print("⚠️ JRA_MONTH_PARAMS not found in scope. Ensure imports are correct.")


✅ JRA 2023/2022 Parameters have been patched successfully.


In [3]:
# 2.1 スクレイパーの修正パッチ (コース詳細・天候取得)
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import scraper.jra_scraper

def patched_scrape_jra_race(url, existing_race_ids=None):
    print(f'Accessing JRA URL (Patched): {url}...')
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.encoding = 'cp932'
        if response.status_code != 200: return None
        soup = BeautifulSoup(response.text, 'html.parser')

        # (Skipping full redundant parse code for brevity in this patch, assuming main logic matches jra_scraper.py)
        # Ideally we should just RELOAD the module if we updated the file in Drive.
        # If the user updates the file in Drive (scraper/jra_scraper.py), they just need to reload.
        # But to be safe, we can force reload.
        pass
    except: pass
    return scraper.jra_scraper.scrape_jra_race(url, existing_race_ids)

import importlib
importlib.reload(scraper.jra_scraper)
from scraper.jra_scraper import scrape_jra_race, scrape_jra_year, JRA_MONTH_PARAMS
print('✅ Loaded updated scraper/jra_scraper.py from Drive')


✅ Loaded updated scraper/jra_scraper.py from Drive


In [4]:
# 3. スクレイピング実行関数の定義

def jra_scrape_execution(year_str, start_date=None, end_date=None):
    CSV_FILE_PATH = os.path.join(PROJECT_PATH, "data", "raw", "database.csv")
    print(f"Using CSV Path: {CSV_FILE_PATH}")

    def save_chunk(df_chunk):
        if os.path.exists(CSV_FILE_PATH):
            try:
                # Read types as string to prevent auto-float for IDs
                existing_df = pd.read_csv(CSV_FILE_PATH, dtype={'race_id': str, 'horse_id': str}, low_memory=False)
                combined_df = pd.concat([existing_df, df_chunk], ignore_index=True)
            except pd.errors.EmptyDataError:
                print("  Note: File empty, creating new.")
                combined_df = df_chunk
            except Exception as e:
                # ★ CRITICAL FIX: Do NOT overwrite on generic read errors (e.g. MemoryError)
                print(f"❌ CRITICAL ERROR: Could not read existing CSV ({e}). Aborting save to prevent data loss.")
                raise e
        else:
            combined_df = df_chunk

        # Deduplication
        subset_cols = ['race_id', '馬名']
        subset_cols = [c for c in subset_cols if c in combined_df.columns]
        if subset_cols:
            combined_df.drop_duplicates(subset=subset_cols, keep='last', inplace=True)

        combined_df.to_csv(CSV_FILE_PATH, index=False, encoding="utf-8-sig")
        print(f"  [Saved] Total rows: {len(combined_df)} (+{len(df_chunk)} new)")

    print(f"Starting Scraping for {year_str} ({start_date} ~ {end_date})")

    # Load existing IDs to skip
    existing_ids = set()
    if os.path.exists(CSV_FILE_PATH):
        try:
             df_e = pd.read_csv(CSV_FILE_PATH, usecols=['race_id'], dtype={'race_id': str}, low_memory=False)
             existing_ids = set(df_e['race_id'].astype(str))
             print(f"  Loaded {len(existing_ids)} existing race IDs to skip.")
        except:
             pass

    scrape_jra_year(year_str, start_date=start_date, end_date=end_date, save_callback=save_chunk, existing_race_ids=existing_ids)


In [ ]:
# 4. 実行パラメータの設定と開始
# -----------------------------
TARGET_YEAR = "2026"
TARGET_MONTH = None  # ★何月を取得するか指定 (Noneの場合は全期間、1〜12を指定)

import calendar
from datetime import date

START_DATE = None
END_DATE = None

if TARGET_MONTH:
    # 指定した月の1日〜末日を設定
    _, last_day = calendar.monthrange(int(TARGET_YEAR), int(TARGET_MONTH))
    START_DATE = date(int(TARGET_YEAR), int(TARGET_MONTH), 1)
    END_DATE = date(int(TARGET_YEAR), int(TARGET_MONTH), last_day)
    print(f"Targeting specific month: {START_DATE} to {END_DATE}")
else:
    # 自動判定ロジック (既存データの翌日から)
    CSV_FILE_PATH = os.path.join(PROJECT_PATH, "data", "raw", "database.csv")
    if os.path.exists(CSV_FILE_PATH):
        try:
             df_exist = pd.read_csv(CSV_FILE_PATH)
             if '日付' in df_exist.columns and not df_exist.empty:
                 df_exist['date_obj'] = pd.to_datetime(df_exist['日付'], format='%Y年%m月%d日', errors='coerce')
                 last_date = df_exist['date_obj'].max()
                 if pd.notna(last_date):
                     # START_DATE = last_date.date() # 旧: 続きから
                     # 新: 欠落補完のために、強制的にその年の1月1日からスキャンする (existing_idsでスキップされる)
                     START_DATE = date(int(TARGET_YEAR), 1, 1)
                     print(f"既存データの最終日時: {last_date.date()} (欠落確認のため {START_DATE} からスキャンします)")
        except Exception as e:
            print(f"既存データ確認エラー: {e}")

print(f"Scraping Target: {TARGET_YEAR}, Start: {START_DATE}, End: {END_DATE}")
jra_scrape_execution(TARGET_YEAR, start_date=START_DATE, end_date=END_DATE)


/tmp/ipython-input-884418207.py:23: DtypeWarning: Columns (5,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_exist = pd.read_csv(CSV_FILE_PATH)


既存データの最終日時: 2026-01-05 (欠落確認のため 2026-01-01 からスキャンします)
Scraping Target: 2026, Start: 2026-01-01, End: None
Using CSV Path: /content/drive/MyDrive/dai-keiba/data/raw/database.csv
Starting Scraping for 2026 (2026-01-01 ~ None)
  Loaded 20781 existing race IDs to skip.
=== Starting JRA Bulk Scraping for 2026 (Period: 2026-01-01 - None) ===
Fetching list for 2026/01 (CNAME=pw01skl00202601/E4)...
  Found 4 race days in month.
    Processing Day: 2026-01-05 (レース結果 レース選択2026年1月5日（月曜）1回中山2日)
      -> 12 races found.
Accessing JRA URL: https://www.jra.go.jp/JRADB/accessS.html?CNAME=pw01sde0106202601020120260105/06...
Skipping 202606010201 (Already exists)
Accessing JRA URL: https://www.jra.go.jp/JRADB/accessS.html?CNAME=pw01sde0106202601020220260105/BB...
Skipping 202606010202 (Already exists)
Accessing JRA URL: https://www.jra.go.jp/JRADB/accessS.html?CNAME=pw01sde0106202601020320260105/70...
Skipping 202606010203 (Already exists)
Accessing JRA URL: https://www.jra.go.jp/JRADB/accessS.html?CNAM

In [ ]:
# 4.1 データベースカラム名の修正 (重複改善)
import os
import pandas as pd

def fix_database_columns_notebook():
    # Google Driveパスを使用
    target_file = os.path.join(PROJECT_PATH, 'data', 'raw', 'database.csv')

    if not os.path.exists(target_file):
        print('Database file not found.')
        return

    print(f'Checking columns in {target_file}...')
    try:
        df = pd.read_csv(target_file, low_memory=False)

        # 表記ゆれ辞書
        columns_to_fix = {
            'コーナー通過順': 'コーナー 通過順',
            '馬体重(増減)': '馬体重 (増減)'
        }

        changed = False
        for bad_col, good_col in columns_to_fix.items():
            if bad_col in df.columns:
                print(f'Merging {bad_col} -> {good_col}...')
                if good_col not in df.columns:
                    df[good_col] = df[bad_col]
                else:
                    df[good_col] = df[good_col].fillna(df[bad_col])

                df.drop(columns=[bad_col], inplace=True)
                changed = True

        if changed:
            df.to_csv(target_file, index=False, encoding='utf-8-sig')
            print('✅ Database columns fixed and saved.')
        else:
            print('No column fixes needed.')

    except Exception as e:
        print(f'Error fixing columns: {e}')

fix_database_columns_notebook()


In [ ]:
# 4.2 欠損データの補完 (HorseID & 過去走 & Metadata & Pedigree)
import pandas as pd
import sys
import os
import re
from datetime import datetime
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

# TQDM progress bar
try:
    from tqdm.auto import tqdm
except ImportError:
    !pip install tqdm
    from tqdm.auto import tqdm

# Ensure scraper path
try:
    from scraper.race_scraper import RaceScraper
except ImportError:
    sys.path.append(os.path.join(PROJECT_PATH, 'scraper'))
    from scraper.race_scraper import RaceScraper

# Global lock for dataframe update
df_lock = threading.Lock()

def fetch_race_horse_ids(rid):
    scraper = RaceScraper()
    try:
        url = f'https://race.netkeiba.com/race/result.html?race_id={rid}'
        soup = scraper._get_soup(url)
        if not soup: return None

        table = soup.find('table', id='All_Result_Table')
        if not table: return None

        horse_map = {}
        rows = table.find_all('tr', class_='HorseList')
        for row in rows:
            name_tag = row.select_one('.Horse_Name a')
            if name_tag:
                h_name = name_tag.text.strip()
                href = name_tag.get('href', '')
                match = re.search(r'/horse/(\d+)', href)
                if match:
                    horse_map[h_name] = match.group(1)
        return (rid, horse_map)
    except Exception as e:
        return None

def fetch_horse_history(horse_id):
    scraper = RaceScraper()
    try:
        df = scraper.get_past_races(str(horse_id), n_samples=None)
        return (horse_id, df)
    except Exception as e:
        return (horse_id, pd.DataFrame())

def fill_missing_past_data_notebook():
    csv_path = os.path.join(PROJECT_PATH, 'data', 'raw', 'database.csv')
    if not os.path.exists(csv_path):
        print(f'Error: {csv_path} not found.')
        return

    print(f'Reading {csv_path}...')
    df = pd.read_csv(csv_path, low_memory=False, dtype={'race_id': str, 'horse_id': str})
    def clean_id_str(x):
        if pd.isna(x) or x == '': return None
        s = str(x)
        if s.endswith('.0'): return s[:-2]
        return s
    if 'race_id' in df.columns:
        df['race_id'] = df['race_id'].apply(clean_id_str)
    if 'horse_id' in df.columns:
        df['horse_id'] = df['horse_id'].apply(clean_id_str)

    if '日付' in df.columns:
        df['date_dt'] = pd.to_datetime(df['日付'], format='%Y年%m月%d日', errors='coerce')
    else:
        print('Error: 日付 column not found.')
        return

    if 'horse_id' not in df.columns:
        df['horse_id'] = None

    # === 1. Fill Missing Horse IDs ===
    if 'race_id' in df.columns:
        missing_mask = df['horse_id'].isna() | (df['horse_id'] == '')
        if missing_mask.any():
            races_to_update = df.loc[missing_mask, 'race_id'].astype(str).unique()
            print(f'Need to fetch IDs for {len(races_to_update)} races...')

            with ThreadPoolExecutor(max_workers=10) as executor:
                futures = {executor.submit(fetch_race_horse_ids, rid): rid for rid in races_to_update}

                for future in tqdm(as_completed(futures), total=len(races_to_update), desc="Fetching IDs"):
                    result = future.result()
                    if result:
                        rid, horse_map = result
                        if horse_map:
                            mask = df['race_id'].astype(str) == str(rid)
                            for h_name, h_id in horse_map.items():
                                h_mask = mask & (df['馬名'] == h_name)
                                if h_mask.any():
                                    df.loc[h_mask, 'horse_id'] = h_id

            df.to_csv(csv_path, index=False, encoding='utf-8-sig')
            print('Saved updated IDs.')
        else:
            print('All Horse IDs present.')

    # === 2. Fill Past History ===
    fields_map = {
        'date': 'date', 'rank': 'rank', 'time': 'time', 'run_style': 'run_style',
        'race_name': 'race_name', 'last_3f': 'last_3f', 'horse_weight': 'horse_weight',
        'jockey': 'jockey', 'condition': 'condition', 'odds': 'odds',
        'weather': 'weather', 'distance': 'distance', 'course_type': 'course_type'
    }

    unique_horses = df['horse_id'].dropna().astype(str).unique()

    # Init columns
    new_cols = []
    for k in fields_map.keys():
        for i in range(1, 6):
            col = f'past_{i}_{k}'
            if col not in df.columns:
                new_cols.append(col)

    if new_cols:
        df_new = pd.DataFrame(None, index=df.index, columns=new_cols, dtype='object')
        df = pd.concat([df, df_new], axis=1)

    print(f'Found {len(unique_horses)} unique horses. Fetching history...')
    history_store = {}

    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = {executor.submit(fetch_horse_history, hid): hid for hid in unique_horses}

        for future in tqdm(as_completed(futures), total=len(unique_horses), desc="Fetching History"):
            hid, hist_df = future.result()
            if not hist_df.empty:
                history_store[str(hid)] = hist_df

    print('Applying history data...')
    for hid in history_store:
        if 'date' in history_store[hid]:
            history_store[hid]['date_obj'] = pd.to_datetime(history_store[hid]['date'], errors='coerce')

    valid_rows = df.dropna(subset=['horse_id', 'date_dt'])

    for idx, row in tqdm(valid_rows.iterrows(), total=len(valid_rows), desc="Applying History"):
        hid = str(row['horse_id'])
        current_date = row['date_dt']

        if hid in history_store:
            hist_df = history_store[hid]
            if hist_df.empty or 'date_obj' not in hist_df.columns: continue

            past_races = hist_df[hist_df['date_obj'] < current_date].sort_values('date_obj', ascending=False).head(5)

            for i, (p_idx, p_row) in enumerate(past_races.iterrows()):
                n = i + 1
                for k, v in fields_map.items():
                    df.at[idx, f'past_{n}_{k}'] = p_row.get(v)

    if 'date_dt' in df.columns: df.drop(columns=['date_dt'], inplace=True)
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print('Done filling past data.')

    # === 3. Fill Blood/Pedigree Data ===
    print('Checking Pedigree Data...')
    ped_cols = ['father', 'mother', 'bms']
    new_ped_cols = [c for c in ped_cols if c not in df.columns]
    if new_ped_cols:
         df_ped = pd.DataFrame(None, index=df.index, columns=new_ped_cols, dtype='object')
         df = pd.concat([df, df_ped], axis=1)

    unique_horses_df = df[['horse_id', 'father']].drop_duplicates('horse_id')
    missing_horses = unique_horses_df[unique_horses_df['father'].isna() | (unique_horses_df['father'] == '')]['horse_id'].dropna().astype(str).unique()

    if len(missing_horses) > 0:
        print(f'Found {len(missing_horses)} horses with missing pedigree. Fetching...')
        ped_map = {}

        def fetch_ped_entry(hid):
            scr = RaceScraper()
            return (hid, scr.get_horse_profile(hid))

        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {executor.submit(fetch_ped_entry, hid): hid for hid in missing_horses}
            for future in tqdm(as_completed(futures), total=len(missing_horses), desc="Fetching Pedigree"):
                try:
                    hid, p_data = future.result()
                    if p_data:
                        ped_map[str(hid)] = p_data
                except: pass

        print('Applying pedigree data...')
        f_map = {h: d.get('father') for h, d in ped_map.items()}
        m_map = {h: d.get('mother') for h, d in ped_map.items()}
        b_map = {h: d.get('bms') for h, d in ped_map.items()}

        df['horse_id_str'] = df['horse_id'].astype(str)
        mask = df['horse_id_str'].isin(ped_map.keys())
        df.loc[mask, 'father'] = df.loc[mask, 'horse_id_str'].map(f_map).fillna(df.loc[mask, 'father'])
        df.loc[mask, 'mother'] = df.loc[mask, 'horse_id_str'].map(m_map).fillna(df.loc[mask, 'mother'])
        df.loc[mask, 'bms'] = df.loc[mask, 'horse_id_str'].map(b_map).fillna(df.loc[mask, 'bms'])

        df.drop(columns=['horse_id_str'], inplace=True, errors='ignore')

        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print('Done filling pedigree.')

    # === 4. Fill Metadata (Course, Dist, Weather) ===
    print('Checking Metadata (Course, Distance, Weather)...')
    meta_cols = ['コースタイプ', '距離', '天候', '馬場状態']
    for c in meta_cols:
        if c not in df.columns: df[c] = None

    missing_meta_mask = (df['コースタイプ'].isna()) | (df['コースタイプ'] == '')
    if 'race_id' in df.columns:
         rids_missing = df.loc[missing_meta_mask, 'race_id'].astype(str).unique()
    else:
         rids_missing = []

    if len(rids_missing) > 0:
        print(f'Found {len(rids_missing)} races with missing metadata. Fetching...')
        meta_results = {}

        def fetch_meta_entry(rid):
            scr = RaceScraper()
            return scr.get_race_metadata(rid)

        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {executor.submit(fetch_meta_entry, rid): rid for rid in rids_missing}

            for future in tqdm(as_completed(futures), total=len(rids_missing), desc="Fetching Metadata"):
                try:
                    data = future.result()
                    if data and data.get('course_type'):
                        meta_results[str(data['race_id'])] = data
                except: pass

        print('Applying metadata...')
        c_map = {rid: d['course_type'] for rid, d in meta_results.items()}
        d_map = {rid: d['distance'] for rid, d in meta_results.items()}
        w_map = {rid: d['weather'] for rid, d in meta_results.items()}
        cond_map = {rid: d['condition'] for rid, d in meta_results.items()}

        df['race_id_str'] = df['race_id'].astype(str)
        mask = df['race_id_str'].isin(meta_results.keys())

        df.loc[mask, 'コースタイプ'] = df.loc[mask, 'race_id_str'].map(c_map).fillna(df.loc[mask, 'コースタイプ'])
        df.loc[mask, '距離'] = df.loc[mask, 'race_id_str'].map(d_map).fillna(df.loc[mask, '距離'])
        df.loc[mask, '天候'] = df.loc[mask, 'race_id_str'].map(w_map).fillna(df.loc[mask, '天候'])
        df.loc[mask, '馬場状態'] = df.loc[mask, 'race_id_str'].map(cond_map).fillna(df.loc[mask, '馬場状態'])

        df.drop(columns=['race_id_str'], inplace=True, errors='ignore')

        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print('Done filling metadata.')
    else:
        print('All metadata present.')

fill_missing_past_data_notebook()


In [ ]:
# 5. 血統データの補完 (Robust Journaling Mode) - JRA版
# 確実性を最優先し、1頭ずつキャッシュファイルに記録しながら進めます。
# 途中で止まっても、再実行時に続きから再開できます。

def fill_jra_bloodline_robust():
    from tqdm.auto import tqdm
    import os
    import csv
    import time

    csv_path = os.path.join(PROJECT_PATH, 'data', 'raw', 'database.csv')
    cache_path = os.path.join(PROJECT_PATH, 'data', 'jra_pedigree_cache.csv')

    if not os.path.exists(os.path.dirname(cache_path)):
        os.makedirs(os.path.dirname(cache_path), exist_ok=True)

    # 1. Load Main Database
    print(f'Reading {csv_path}...')
    df = pd.read_csv(csv_path, low_memory=False, dtype={'race_id': str, 'horse_id': str})

    if 'horse_id' in df.columns:
        df['horse_id'] = df['horse_id'].astype(str).str.replace(r'\.0$', '', regex=True)

    # 2. Load/Create Cache
    # cache format: horse_id,father,mother,bms
    cached_ids = set()
    if os.path.exists(cache_path):
        try:
            cache_df = pd.read_csv(cache_path, dtype=str)
            cached_ids = set(cache_df['horse_id'].unique())
            print(f"Loaded {len(cached_ids)} cached records from {os.path.basename(cache_path)}")
        except:
            print("Cache file corrupted or empty. Starting fresh.")
    else:
        # Initialize cache file with header
        with open(cache_path, 'w', encoding='utf-8') as f:
            f.write('horse_id,father,mother,bms\n')

    # 3. Identify Missing Horses
    # Check if columns exist
    for col in ['father', 'mother', 'bms']:
        if col not in df.columns: df[col] = None

    # Missing in DB AND Not in Cache
    mask_missing_db = (df['father'].isna()) | (df['father'] == '') | (df['father'] == 'nan')
    missing_ids = df.loc[mask_missing_db, 'horse_id'].dropna().unique()
    missing_ids = [h for h in missing_ids if h.isdigit()]

    # Filter out already cached
    target_horses = [h for h in missing_ids if h not in cached_ids]

    print(f"Total Missing in DB: {len(missing_ids)}")
    print(f"Already Cached: {len(cached_ids)}")
    print(f"Target to Scrape: {len(target_horses)}")

    if len(target_horses) > 0:
        from scraper.race_scraper import RaceScraper
        scraper = RaceScraper()

        print("Starting Row-by-Row Scraping...")

        # Append mode for safety
        with open(cache_path, 'a', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)

            # Disable threads for maximum safety/stability if requested, or use simple loop
            # User asked for 'Try before implement' and 'One by one'.
            # Single thread loop is safest for 'One by one writing'.

            for i, hid in enumerate(tqdm(target_horses)):
                try:
                    profile = scraper.get_horse_profile(hid)
                    if profile:
                        father = profile.get('father', '')
                        mother = profile.get('mother', '')
                        bms = profile.get('bms', '')

                        writer.writerow([hid, father, mother, bms])
                        f.flush() # Ensure write to disk immediately
                except Exception as e:
                    print(f"Error scraping {hid}: {e}")
                    # Continue to next

    else:
        print("No new horses to scrape.")

    # 4. Merge Cache into Database
    print("Merging cache into database...")
    if os.path.exists(cache_path):
        cache_df = pd.read_csv(cache_path, dtype=str)

        # Convert to dict for mapping
        f_map = dict(zip(cache_df['horse_id'], cache_df['father']))
        m_map = dict(zip(cache_df['horse_id'], cache_df['mother']))
        b_map = dict(zip(cache_df['horse_id'], cache_df['bms']))

        # Apply map
        mask = df['horse_id'].isin(f_map.keys())
        df.loc[mask, 'father'] = df.loc[mask, 'horse_id'].map(f_map).fillna(df.loc[mask, 'father'])
        df.loc[mask, 'mother'] = df.loc[mask, 'horse_id'].map(m_map).fillna(df.loc[mask, 'mother'])
        df.loc[mask, 'bms'] = df.loc[mask, 'horse_id'].map(b_map).fillna(df.loc[mask, 'bms'])

        # Save DB
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print("✅ Database updated successfully.")

fill_jra_bloodline_robust()

# 5. 血統データの補完 (Bloodline Backfill)

取得漏れの血統データ（父、母、母父）をバックグラウンドで補完します。
既存のデータを確認し、欠損がある馬のみ対象とします。

In [ ]:
# 血統データのバックフィル実行
from scraper.auto_scraper import fill_bloodline_data

print("Starting Bloodline Backfill process...")
fill_bloodline_data(mode="JRA")
print("Bloodline Backfill completed.")